In [2]:
import torch

# 📘S1W4D2: 引用与内存 (References & Memory)

## 1 核心概念：看不见的“线”

**底层联系 (The Connection):**

  * **链表 (Linked List)**：节点之间通过指针（引用）连接。丢失了指针，就丢失了整个链表。
  * **计算图 (Computational Graph)**：PyTorch 的自动微分图本质上就是一个巨大的、复杂的链表（DAG）。`Loss` 顺着 `grad_fn` 这根线往回找，才能更新 `Weights`。如果中间这根线断了（比如误用了 `detach`），模型就没法学习。

## 2 PyTorch 内存实战 (The "Job" Skills)

在训练模型时，我们经常需要复制张量。但是，怎么复制决定了梯度能不能传过去。

### 2.1 引用 vs 拷贝 (Reference vs Copy)

请运行以下代码，观察内存地址变化：

In [ ]:
# 1. 基础引用 (Python 默认行为)
x = torch.tensor([1.0, 2.0, 3.0,], requires_grad = True)
y = x

print(f"x 的内存地址: {id(x)}")
print(f"y 的内存地址: {id(y)}")
print(f"x 和 y 是同一个对象吗? {x is y}") # True

# 修改 y 会影响 x
with torch.no_grad():
    y[0] = 100
print(f"修改 y 后 x 的值: {x}") # x 也变了

### 2.2 Clone vs Detach (面试必问)

这是今天最重要的 PyTorch 知识点。

  * **`clone()`**: 复制数据，**保留**梯度链。用于“我要复制一份数据参与计算，且希望反向传播能传回来”。
  * **`detach()`**: 共享/复制数据（视情况而定），**切断**梯度链。用于“我要把 tensor 拿出来画图、保存或做推理，不想影响原来的计算图”。


In [7]:
# 初始化
a = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
b = a * 2 

# --- 实验 1: .clone() ---
# c 是 b 的克隆。c 参与计算，梯度会传回 b，也会传回 a
c = b.clone()
d = c * 3
d.sum().backward()

print(f"a 的梯度 (expect 6.0): {a.grad}") 
# 推导: d = c*3 = b*3 = (a*2)*3 = 6a -> d/da = 6

# --- 清空梯度 ---
a.grad.zero_()

# --- 实验 2: .detach() (修正版) ---

# e 从 b 分离出来。
# 此时 e 是一个新的 Leaf Node（叶子节点），它和 b 共享数值，但没有梯度连接。
e = b.detach()

# 【关键修正】：为了能对 f 调用 backward，我们必须让 e 开始追踪梯度。
# 这模拟了 e 作为"新网络的输入"的场景。
e.requires_grad_(True)

f = e * 3
f.sum().backward() 

# 验证结果
print(f"e 的梯度 (expect 3.0): {e.grad}") # e 自己是一个新的源头，它能收到梯度
print(f"Detach 后 a 的梯度 (expect 0.0): {a.grad}") # 但是！梯度传不到 a 那里去

# 结论：detach() 就像一把剪刀，剪断了 b 和 e 之间的反向传播路径。

a 的梯度 (expect 6.0): tensor([6., 6., 6.])
e 的梯度 (expect 3.0): tensor([3., 3., 3.])
Detach 后 a 的梯度 (expect 0.0): tensor([0., 0., 0.])


> **🛠️ 工程场景**:
>
>   * **Transfer Learning (迁移学习)**: 冻结骨干网络时，会用到 `detach()` 或者设置 `requires_grad=False`。
>   * **Reinforcement Learning (强化学习)**: Target Network 更新通常需要 `detach()`。

## 3 LeetCode 算法突击 (The "Interview" Skills)

链表题不考复杂的逻辑，只考你\*\*“心细”\*\*。所有的坑都在于：指针指歪了、断了、或者死循环了。

为了在 Python 中练习链表，我们需要先定义一个简单的节点类（LeetCode 也是这样定义的）：

In [8]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

# 辅助函数：将列表转为链表 (方便测试)
def create_linked_list(arr):
    if not arr: return None
    head = ListNode(arr[0])
    curr = head
    for val in arr[1:]:
        curr.next = ListNode(val)
        curr = curr.next
    return head

# 辅助函数：打印链表
def print_linked_list(head):
    vals = []
    curr = head
    while curr:
        vals.append(str(curr.val))
        curr = curr.next
    print(" -> ".join(vals))

### 3.1: 反转链表 (Reverse Linked List) - Easy

  * **LeetCode 链接**: [206. Reverse Linked List](../../LeetCode%20practice/201-250.ipynb)
  * **AI 关联**: 类似于时间序列的反向传播（BPTT），我们需要从后往前推导。
  * **核心思路**: **双指针/三指针**。需要 `prev` (前一个), `curr` (当前), `next_temp` (先存下个节点，不然一断链就找不到了)。

In [9]:
def reverseList(head):
    prev = None
    curr = head
    
    while curr:
        # 1. 暂存下一步要去的地方 (关键！否则链条断了就回不来了)
        next_temp = curr.next 
        
        # 2. 调转箭头方向
        curr.next = prev
        
        # 3. 整体向后移动一步
        prev = curr
        curr = next_temp
        
    return prev # prev 变成了新的头节点

# 测试
head = create_linked_list([1, 2, 3, 4, 5])
print("原始链表:")
print_linked_list(head)

new_head = reverseList(head)
print("反转后:")
print_linked_list(new_head)

原始链表:
1 -> 2 -> 3 -> 4 -> 5
反转后:
5 -> 4 -> 3 -> 2 -> 1


### 3.2 环形链表 (Linked List Cycle) - Easy

  * **LeetCode 链接**: [141. Linked List Cycle](../../LeetCode%20practice/101-150.ipynb)
  * **AI 关联**: **死循环检测**。训练过程中的 Loss 不下降或者震荡，就像是陷入了一个“环”。
  * **核心思路**: **快慢指针 (Fast & Slow Pointers)**。
      * `Slow` 跑一步，`Fast` 跑两步。
      * 如果没环，Fast 会先跑到终点（None）。
      * 如果有环，Fast 最终会从后面“套圈”追上 Slow。

In [10]:
def hasCycle(head):
    if not head or not head.next:
        return False
    
    slow = head
    fast = head.next
    
    while slow != fast:
        # 如果 fast 跑到头了，说明没环
        if not fast or not fast.next:
            return False
        
        slow = slow.next      # 慢跑 1 步
        fast = fast.next.next # 快跑 2 步
        
    return True # 相遇了，说明有环

# --- 构造一个带环的链表测试 ---
# 1 -> 2 -> 3 -> 4
#           ^    |
#           |____|
node1 = ListNode(1)
node2 = ListNode(2)
node3 = ListNode(3)
node4 = ListNode(4)

node1.next = node2
node2.next = node3
node3.next = node4
node4.next = node2 # 4 指回 2，形成环

print(f"\n链表是否有环? {hasCycle(node1)}")


链表是否有环? True


## 4 总结与作业

### 今日复盘

1.  **PyTorch 内存**:
      * `clone()` = 影分身（带连接，能传梯度）。
      * `detach()` = 断绝关系（不带连接，不传梯度）。
2.  **算法技巧**:
      * 链表操作核心是 **"先存后改"**（在改变 `curr.next` 之前，务必先存下 `next_temp`）。
      * 快慢指针是解决循环问题的神器。

### 课后作业

1.  在 Notebook 中运行以上代码，重点画图理解 `reverseList` 的过程（可以用纸笔画箭头辅助）。

2.  **思考题**: 为什么我们在 `src/engine.py` 的训练循环里，计算准确率时通常要写：
    `preds = torch.argmax(logits, dim=1).cpu().numpy()`
    这里如果不加 `.detach()` (通常转 numpy 会自动 detach，但如果是显式操作呢)，会有什么风险？

    *(答案提示：如果把带梯度的 Tensor 存到列表里一直不释放，会导致计算图越来越大，最后撑爆内存。)*